In [21]:
import glob
import json
import create_dataset
import os
import sys
from shutil import copy
from pathlib import Path
from random import shuffle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [22]:
%matplotlib inline

In [23]:
base_image_directory = '/media/mehdi2277/MyFiles/large_datasets/text_classification/Test_Sanborn_Words/'
image_directory = '/media/mehdi2277/MyFiles/large_datasets/text_classification/Test_Sanborn_Words/Skewed_Numbers/'
word_image_paths = glob.glob(image_directory + '*.png')

In [4]:
text_labels_file_name = 'text_labels_sanborn_testing.json'
text_labels_file_handle = open(base_image_directory + text_labels_file_name, 'r')
word_image_labels = json.load(text_labels_file_handle)
text_labels_file_handle.close()

In [5]:
prefix_len = len('/media/mehdi2277/MyFiles/large_datasets/text_classification/Test_Sanborn_Words/')
suffix_word_image_labels = {k[prefix_len:]: v for k,v in word_image_labels.items()}
word_image_labels = {image_directory + k: v for k,v in suffix_word_image_labels.items()}

In [24]:
text_labels_file_name2 = 'text_labels_sanborn_testing2.json'
text_labels_file_handle2 = open(base_image_directory + text_labels_file_name2, 'r')
word_image_labels2 = json.load(text_labels_file_handle2)
text_labels_file_handle2.close()

In [25]:
word_image_labels2 = {image_directory + k: v for k,v in word_image_labels2.items()}

In [26]:
full_word_image_labels = {}
word_image_labels = {}
full_word_image_labels.update(word_image_labels)
full_word_image_labels.update(word_image_labels2)

In [33]:
present_word_image_paths = list(filter(lambda word: word in full_word_image_labels, word_image_paths))

In [28]:
home_directory = os.path.expanduser('~')
nn_library_path = home_directory + '/Documents/HarveyMuddWork/Neural_Nets_Research/neural_nets_research/neural_nets_library'
sys.path.append(nn_library_path)

In [29]:
from dataset import greyscale_image_loader, tightest_image_crop, square_padding
from torchvision import transforms
from PIL import Image

In [30]:
def vertical_scale_preserve_aspect_ratio(img, height=32):
    w, h = img.size
    
    return transforms.Resize((height,int(height/h * w)), interpolation=Image.NEAREST)(img)

In [31]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(tightest_image_crop),
        transforms.ToPILImage(),
        transforms.Lambda(vertical_scale_preserve_aspect_ratio),
    ])

In [15]:
for image_path in word_image_paths:
    datapoint = greyscale_image_loader(image_path)
    image = transform(datapoint)
    image.save(image_path)

In [34]:
word_image_paths_by_width = sorted(present_word_image_paths, key=lambda p: greyscale_image_loader(p).size[0])
word_image_labels_by_width = [full_word_image_labels[path] for path in word_image_paths_by_width]

In [35]:
database_file_name = '/media/mehdi2277/MyFiles/large_datasets/text_classification/lmdb_files/skewed_sanborn_numbers_test.mdb'

In [36]:
create_dataset.createDataset(database_file_name, word_image_paths_by_width, word_image_labels_by_width)

Created dataset with 500 samples
